In [24]:
import numpy as np

n_timestamps = 10
X, y = np.arange(n_timestamps), np.arange(n_timestamps) - n_timestamps
X, y

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([-10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1]))

In [25]:
from gtda.time_series import SlidingWindow

window_size = 3
stride = 2

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)
X_sw, yr

(array([[1, 2, 3],
        [3, 4, 5],
        [5, 6, 7],
        [7, 8, 9]]),
 array([-7, -5, -3, -1]))

In [26]:
rng = np.random.default_rng(42)

n_features = 2

X = rng.integers(0, high=20, size=(n_timestamps, n_features), dtype=int)
X

array([[ 1, 15],
       [13,  8],
       [ 8, 17],
       [ 1, 13],
       [ 4,  1],
       [10, 19],
       [14, 15],
       [14, 15],
       [10,  2],
       [16,  9]])

In [27]:
SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)
X_sw, yr

(array([[[13,  8],
         [ 8, 17],
         [ 1, 13]],
 
        [[ 1, 13],
         [ 4,  1],
         [10, 19]],
 
        [[10, 19],
         [14, 15],
         [14, 15]],
 
        [[14, 15],
         [10,  2],
         [16,  9]]]),
 array([-7, -5, -3, -1]))

In [28]:
from gtda.time_series import PearsonDissimilarity
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude

PD = PearsonDissimilarity()
X_pd = PD.fit_transform(X_sw)
VR = VietorisRipsPersistence(metric="precomputed")
X_vr = VR.fit_transform(X_pd)  # "precomputed" required on dissimilarity data
Ampl = Amplitude()
X_a = Ampl.fit_transform(X_vr)
X_a

array([[0.18228669, 0.        ],
       [0.03606068, 0.        ],
       [0.28866041, 0.        ],
       [0.01781238, 0.        ]])

In [29]:
from sklearn import set_config
set_config(display='diagram')  # For HTML representations of pipelines

from gtda.pipeline import make_pipeline

pipe = make_pipeline(SW, PD, VR, Ampl)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=3, stride=2)),
                ('pearsondissimilarity', PearsonDissimilarity()),
                ('vietorisripspersistence',
                 VietorisRipsPersistence(metric='precomputed')),
                ('amplitude', Amplitude())])

In [30]:
from sklearn.ensemble import RandomForestRegressor

RFR = RandomForestRegressor()

pipe = make_pipeline(SW, PD, VR, Ampl, RFR)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=3, stride=2)),
                ('pearsondissimilarity', PearsonDissimilarity()),
                ('vietorisripspersistence',
                 VietorisRipsPersistence(metric='precomputed')),
                ('amplitude', Amplitude()),
                ('randomforestregressor', RandomForestRegressor())])

In [31]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

/Users/ricardo/miniconda3/envs/gtda/lib/python3.9/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning:

if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.



(array([-5.84, -4.  , -4.08, -2.24]), 0.74752)

In [32]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=X[:, 0], y=y, mode="markers", name="data"))
fig.add_trace(
    go.Scatter(x=X[:, 0], y=y_pred, mode="markers", name="prediction")
)
fig.show()



In [33]:
import pandas as pd

In [34]:
# Read the data from the NOAA website
df = pd.read_fwf('https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt', colspecs="infer")

In [35]:
# Read the data from the NOAA website
df = pd.read_fwf('https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt', colspecs="infer")
# Group by 'Year' and add a sequential month number starting from 1
df['MONTH'] = df.groupby('YR').cumcount() + 1
# add a leading zero to the month number if it is less than 2 digits
df['MONTH'] = df['MONTH'].apply(lambda x: '{0:0>2}'.format(x))

In [36]:
# Read the data from the NOAA website
df = pd.read_fwf('https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt', colspecs="infer")
# Group by 'Year' and add a sequential month number starting from 1
df['MONTH'] = df.groupby('YR').cumcount() + 1
# add a leading zero to the month number if it is less than 2 digits
df['MONTH'] = df['MONTH'].apply(lambda x: '{0:0>2}'.format(x))
# create a new column 'DATE' by concatenating the 'YR' and 'MONTH' columns
df['DATE'] = df['YR'].astype(str) + '-' + df['MONTH'].astype(str)
# drop the 'YR' and 'MONTH' columns and convert the 'DATE' column to datetime format
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m')
# Create a new dataframe with necessary columns
data = df[['DATE', 'TOTAL', 'ANOM']].copy()
data.columns = ['Date', 'Temperature', 'Anomaly']
data

,Date,Temperature,Anomaly
0,1950-01-01,24.72,-1.53
1,1950-02-01,25.17,-1.34
2,1950-03-01,25.75,-1.16
3,1950-04-01,26.12,-1.18
4,1950-05-01,26.32,-1.07
...,...,...,...
874,2022-11-01,25.76,-0.92
875,2022-12-01,25.80,-0.82
876,2023-01-01,25.97,-0.67
877,2023-02-01,26.44,-0.43


In [37]:
data['Temperature Y'] = data['Temperature'].shift(-1)
data['Anomaly Y'] = data['Anomaly'].shift(-1)
data = data.drop(data.tail(1).index) # Drop last row
data.reset_index(drop = True, inplace = True)
data

,Date,Temperature,Anomaly,Temperature Y,Anomaly Y
0,1950-01-01,24.72,-1.53,25.17,-1.34
1,1950-02-01,25.17,-1.34,25.75,-1.16
2,1950-03-01,25.75,-1.16,26.12,-1.18
3,1950-04-01,26.12,-1.18,26.32,-1.07
4,1950-05-01,26.32,-1.07,26.31,-0.85
...,...,...,...,...,...
873,2022-10-01,25.72,-0.99,25.76,-0.92
874,2022-11-01,25.76,-0.92,25.80,-0.82
875,2022-12-01,25.80,-0.82,25.97,-0.67
876,2023-01-01,25.97,-0.67,26.44,-0.43


## TOTAL TOPOLOGICAL FORECASTING

In [38]:
X1 = data['Temperature'].to_numpy()
y1 = data['Temperature Y'].to_numpy()
X1, y1

(array([24.72, 25.17, 25.75, 26.12, 26.32, 26.31, 26.21, 25.96, 25.76,
        25.63, 25.48, 25.34, 25.42, 25.96, 26.74, 27.48, 27.75, 27.75,
        27.44, 27.28, 27.14, 27.22, 27.12, 26.95, 26.78, 26.87, 27.25,
        27.6 , 27.59, 27.17, 26.67, 26.39, 26.3 , 26.17, 26.13, 26.29,
        26.65, 27.1 , 27.53, 27.96, 28.14, 27.94, 27.49, 27.12, 26.93,
        26.91, 26.92, 26.95, 27.  , 26.97, 26.86, 26.89, 26.85, 26.67,
        26.1 , 25.54, 25.25, 25.3 , 25.35, 25.48, 25.57, 25.89, 26.22,
        26.5 , 26.6 , 26.45, 26.06, 25.64, 25.06, 24.65, 24.41, 24.72,
        25.23, 25.86, 26.37, 26.82, 26.93, 26.72, 26.23, 25.87, 25.71,
        25.67, 25.67, 25.77, 26.09, 26.68, 27.41, 28.08, 28.37, 28.34,
        28.06, 27.75, 27.5 , 27.48, 27.64, 27.94, 28.15, 28.27, 28.26,
        28.28, 28.19, 27.87, 27.38, 26.85, 26.56, 26.53, 26.6 , 26.81,
        26.95, 27.24, 27.51, 27.68, 27.65, 27.16, 26.63, 26.15, 26.08,
        26.07, 26.15, 26.16, 26.24, 26.51, 26.92, 27.39, 27.47, 27.26,
      

In [39]:
from gtda.time_series import SlidingWindow
from gtda.time_series import SingleTakensEmbedding


STE = SingleTakensEmbedding(parameters_type="search", time_delay=2, dimension=3)
X_ste, yr = STE.fit_transform_resample(X1, y1)
X_ste, yr

(array([[24.72, 25.75],
        [25.17, 26.12],
        [25.75, 26.32],
        ...,
        [25.72, 25.8 ],
        [25.76, 25.97],
        [25.8 , 26.44]]),
 array([26.12, 26.32, 26.31, 26.21, 25.96, 25.76, 25.63, 25.48, 25.34,
        25.42, 25.96, 26.74, 27.48, 27.75, 27.75, 27.44, 27.28, 27.14,
        27.22, 27.12, 26.95, 26.78, 26.87, 27.25, 27.6 , 27.59, 27.17,
        26.67, 26.39, 26.3 , 26.17, 26.13, 26.29, 26.65, 27.1 , 27.53,
        27.96, 28.14, 27.94, 27.49, 27.12, 26.93, 26.91, 26.92, 26.95,
        27.  , 26.97, 26.86, 26.89, 26.85, 26.67, 26.1 , 25.54, 25.25,
        25.3 , 25.35, 25.48, 25.57, 25.89, 26.22, 26.5 , 26.6 , 26.45,
        26.06, 25.64, 25.06, 24.65, 24.41, 24.72, 25.23, 25.86, 26.37,
        26.82, 26.93, 26.72, 26.23, 25.87, 25.71, 25.67, 25.67, 25.77,
        26.09, 26.68, 27.41, 28.08, 28.37, 28.34, 28.06, 27.75, 27.5 ,
        27.48, 27.64, 27.94, 28.15, 28.27, 28.26, 28.28, 28.19, 27.87,
        27.38, 26.85, 26.56, 26.53, 26.6 , 26.81, 26.95, 27.

In [40]:
window_size = 5
stride = 2

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X_ste, yr)
X_sw, yr

(array([[[25.17, 26.12],
         [25.75, 26.32],
         [26.12, 26.31],
         [26.32, 26.21],
         [26.31, 25.96]],
 
        [[26.12, 26.31],
         [26.32, 26.21],
         [26.31, 25.96],
         [26.21, 25.76],
         [25.96, 25.63]],
 
        [[26.31, 25.96],
         [26.21, 25.76],
         [25.96, 25.63],
         [25.76, 25.48],
         [25.63, 25.34]],
 
        ...,
 
        [[26.62, 26.8 ],
         [26.84, 26.48],
         [26.8 , 26.04],
         [26.48, 25.75],
         [26.04, 25.72]],
 
        [[26.8 , 26.04],
         [26.48, 25.75],
         [26.04, 25.72],
         [25.75, 25.76],
         [25.72, 25.8 ]],
 
        [[26.04, 25.72],
         [25.75, 25.76],
         [25.72, 25.8 ],
         [25.76, 25.97],
         [25.8 , 26.44]]]),
 array([25.76, 25.48, 25.42, 26.74, 27.75, 27.44, 27.14, 27.12, 26.78,
        27.25, 27.59, 26.67, 26.3 , 26.13, 26.65, 27.53, 28.14, 27.49,
        26.93, 26.92, 27.  , 26.86, 26.85, 26.1 , 25.25, 25.35, 25.57,
    

In [41]:
import xgboost as xgb


VR = VietorisRipsPersistence()  # No "precomputed" for point clouds
Ampl = Amplitude()

reg = RandomForestRegressor()

pipe = make_pipeline(STE, SW, VR, Ampl, reg)
pipe

Pipeline(steps=[('singletakensembedding',
                 SingleTakensEmbedding(dimension=3, time_delay=2)),
                ('slidingwindow', SlidingWindow(size=5, stride=2)),
                ('vietorisripspersistence', VietorisRipsPersistence()),
                ('amplitude', Amplitude()),
                ('randomforestregressor', RandomForestRegressor())])

In [42]:
pipe.fit(X1, y1)
y_pred = pipe.predict(X1)
score = pipe.score(X1, y1)
score

/Users/ricardo/miniconda3/envs/gtda/lib/python3.9/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning:

if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.



0.8193631499971601

In [53]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=, y=y1, mode="lines", name="TOTAL"))
fig.add_trace(go.Scatter(x=[i for i in range(len(y_pred), len(data.index))], y=y_pred, mode="lines", name="pred"))


fig.show()

In [54]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=[i for i in range(len(y_pred))], y=y1, mode="lines", name="TOTAL"))
fig.add_trace(go.Scatter(x=[i for i in range(len(y_pred))], y=y_pred, mode="lines", name="pred"))


fig.show()

In [55]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=X1, y=y1, mode="lines", name="TOTAL"))


fig.show()

In [45]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=5, test_size=24*365*1, gap=24)
df = df.sort_index()